In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("motogp")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=motogp>


In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

In [3]:
events = spark.read.csv("DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                        header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")
                        

In [4]:
events.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Parent_sys_id: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Mentions: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Name_source: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Pub_date: timestamp (nullable = true)
 |-- URLs: string (nullable = true)
 |-- Tipe_action: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Has_link: byte (nullable = true)
 |-- Has_picture: byte (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Activity: long (nullable = true)
 |-- Followers: long (nullable = true)
 |-- Following: long (nullable = true)
 |-- Location: string (nullable = true)



## Consulta 1

Calcular el número total de tweets por usuario para los 150 usuarios con más mensajes en la muestra de tweet analizados. Calcular para cada uno de estos usuarios la media de hastags enviados por mensaje.

### Consulta 1a

En esta consulta vamos a obtener los 150 usuarios con más mensajes.

In [5]:
los150 = (events.select("Source", "Id")
    .groupBy("Source")
    .agg(count("Id").alias("tweets"))
    .orderBy("tweets", ascending=False)
    .limit(150))

In [6]:
los150.show()

+---------------+------+
|         Source|tweets|
+---------------+------+
|     m_azharaji|   486|
|     twitMOTOGP|   401|
|      johnbokke|   297|
|   qatarflights|   283|
|     box_repsol|   267|
|    yolandaa_95|   185|
|    AlessiaPont|   182|
|     MM93Lovers|   169|
|    motomatters|   169|
|     Sonic_Moto|   165|
|  noelia_260797|   157|
|    birtymotogp|   155|
|  trackseven707|   152|
|   crash_motogp|   149|
|MarcMarquezTeam|   148|
|    pedrosistas|   148|
|   MotoFamilyGP|   144|
|   Kay46_MotoGP|   142|
|     Ciintiia93|   133|
|    blogenboxes|   133|
+---------------+------+
only showing top 20 rows



## Consulta 1b

Ahora para este grupo de usuarios, vamos a calcular la media de hastag enviandos, para ello vamos calcular cuantos hastag envias los usuarios y realizando un join por la columna Source, nos quedaremos con los tweets enviados por esos 150 usuarios.

El procedimiento a seguir será devidir el body en palabras y contar cuantas de ella empiezan por # lo cual implica que es un hastag

Para pasar de una columna de listas a una columna con estas filas desglosadas usaremos explode.

In [7]:
j = events.select('Source', 'Body').withColumn('k', split(events.Body, ' '))

In [8]:
j2 = j.select(col("k"), explode(col("k")).alias("m"), col('Source'))

In [9]:
j3 = j2.filter(j2.m.rlike('^#'))

In [10]:
j4 = j3.groupBy('Source').agg(count('m').alias('total'))

Aqui ya podemos ver el número de hastag que envia cada usuario.

In [11]:
j4.show()

+---------------+-----+
|         Source|total|
+---------------+-----+
|          heyst|    2|
|   threesnaGINT|    1|
|TheLittleNobody|    2|
|     speed_soul|   13|
|    haydncobb22|    6|
|   racytracey77|    4|
| Etienne_Dokkum|   24|
|monze_cervantes|    2|
|      Rodando26|   10|
|  GeraldohDiiaz|    2|
|     kissdawind|    1|
|      AidenGunn|    1|
|    carl_watton|    1|
|     eloyete_cg|   12|
|  albertomordoj|    1|
|      alexf1man|   17|
|  HelenYatesArt|    4|
|    RafaRamos_9|    1|
|      KaiserD13|    2|
|   Saritacosita|    1|
+---------------+-----+
only showing top 20 rows



In [12]:
distribuccion_final = (j4.alias('a').join(los150.alias('b'), j4.Source == los150.Source)
                       .select('a.Source', 'total', 'tweets') )

In [13]:
media = distribuccion_final.withColumn('media', distribuccion_final.total/distribuccion_final.tweets)

In [14]:
media.show()

+---------------+-----+------+------------------+
|         Source|total|tweets|             media|
+---------------+-----+------+------------------+
|     Tom_Cruis3|  112|    56|               2.0|
|    Noeliiia_93|   55|    70|0.7857142857142857|
|         MotoGP|   84|    95|0.8842105263157894|
|    Martin_Fi3d|  110|    55|               2.0|
|    marianav3ga|  116|    58|               2.0|
|  trackseven707|  136|   152|0.8947368421052632|
|        MiniD0g|  116|    58|               2.0|
|       Wyndha14|   48|    66|0.7272727272727273|
| nisaauliarahma|   64|    64|               1.0|
|      JesiPacio|  132|   130|1.0153846153846153|
|    VAVEL_motor|   89|   124| 0.717741935483871|
|     Javimoto46|   45|    55|0.8181818181818182|
|   MotoGPquotes|  159|   123|1.2926829268292683|
|   dinaarifiani|   42|    75|              0.56|
|Silverstonecabs|  156|    78|               2.0|
|  Alice_Inferno|  118|    59|               2.0|
|   adelyellow46|   34|    78|0.4358974358974359|


In [15]:
media.orderBy('media').show()

+---------------+-----+------+-------------------+
|         Source|total|tweets|              media|
+---------------+-----+------+-------------------+
|       Cev_Ford|    6|   115|0.05217391304347826|
|    motomatters|   18|   169|0.10650887573964497|
|    MartinoMoto|   14|    96|0.14583333333333334|
|    pedrosistas|   25|   148|0.16891891891891891|
|    FansHM_9323|   19|   107|0.17757009345794392|
|MarcMarquezTeam|   27|   148|0.18243243243243243|
|MOTOaceleracion|   15|    71| 0.2112676056338028|
| ArrancaMotores|   15|    61| 0.2459016393442623|
|  ArwikaYulenda|   21|    84|               0.25|
| ERA_agustini93|   25|    86|0.29069767441860467|
|    ainoaaaaa95|   18|    58| 0.3103448275862069|
|        aryVR46|   25|    72| 0.3472222222222222|
|     Judith5Fdz|   21|    57| 0.3684210526315789|
|   93MarquezFan|   26|    68|0.38235294117647056|
|   Kay46_MotoGP|   57|   142| 0.4014084507042254|
|        deabruX|   36|    87|0.41379310344827586|
|   adelyellow46|   34|    78| 

In [16]:
media.count()

145

Faltan 5 de esos 150, será debido ha que no tienen hastag, es decir, de media 0

Como se aprecia realizando el orderBy cuando no tenemos hastag no aparace

# Consulta 2


Buscamos aquellos tweets que tienen información de geolocaliación, la condición que deben cumplir es que comienze el campo  por ÜT.  Por tanto, filtrando por esta información y relizando un count sobre los que cumplen esta codición tendríamos resulta la consulta.

Antes de empezar vamos a revisar esta columna.

In [19]:
events.select('Location').show()

+-------------------+
|           Location|
+-------------------+
|          ???????? |
|          ???????? |
|          ???????? |
|          ???????? |
|          ???????? |
|          ???????? |
|London and Slovenia|
|   IGD-Bangsal-ICCU|
|             España|
|         not public|
|         not public|
|      Sumbawa Barat|
|         not public|
|         Only in TV|
|         Only in TV|
|    Brescia, Italia|
|             España|
|             España|
|         not public|
|         not public|
+-------------------+
only showing top 20 rows



Localización

In [20]:
localizados = events.select('Location').filter(events.Location.rlike('^ÜT'))

In [21]:
localizados.count()

2136

In [22]:
localizados.show()

+--------------------+
|            Location|
+--------------------+
|ÜT: -6.194637,106...|
|ÜT: -6.9438429,10...|
|ÜT: 13.750438,100...|
|ÜT: 13.750438,100...|
|ÜT: 52.338471,-0....|
|ÜT: -8.5418039,11...|
|ÜT: -8.5418039,11...|
|ÜT: -8.5418039,11...|
|ÜT: -6.9438429,10...|
| ÜT: -8.015,112.6285|
|ÜT: -6.9391193,10...|
|ÜT: -8.5418039,11...|
|ÜT: -8.5418039,11...|
|ÜT: -6.21113,106....|
|ÜT: 0.51545,101.4...|
|ÜT: 52.338471,-0....|
|ÜT: -6.814538,107...|
|ÜT: -6.1314834,10...|
|ÜT: 4.624875,-74....|
|ÜT: 10.0678745,-6...|
+--------------------+
only showing top 20 rows



Como vemos el número de tweets geolozalizados es de 2136.

## CONSULTA 3

Calcular las 5 cuentas de tweeter más mencionadas, para ello filtramos aquellas que tienen tipe_action == 'MT' ya que en caso contario estariamos introduciendo información de otro tipo de acciones que no nos interesan.

In [23]:
grupo3 = (events.select('Source', 'Mentions', 'Tipe_action')
          .filter(events.Tipe_action == 'MT')
          .withColumn('k', split(events.Mentions, ',')))

Con este paso hemos creado una columna k en la cual ya tenemos una lista con las cuentas que se mencionan en cada tweet.

In [24]:
metions = grupo3.select(col("k"), explode(col("k")).alias("m"))

Realizando el explode sobre esa columna generando otra nueva llamada m, ya tenemos todas las menciones, si ahora las constamos ya tendriamos solucionado la consulta.

In [25]:
metions.show()

+--------------------+---------------+
|                   k|              m|
+--------------------+---------------+
|[marcmarquez93, v...|  marcmarquez93|
|[marcmarquez93, v...|   valeyellow46|
|             [aan__]|          aan__|
|     [marcmarquez93]|  marcmarquez93|
|     [marcmarquez93]|  marcmarquez93|
|     [marcmarquez93]|  marcmarquez93|
|[rossistas, valey...|      rossistas|
|[rossistas, valey...|   valeyellow46|
|      [valeyellow46]|   valeyellow46|
|    [valeyellow46, ]|   valeyellow46|
|    [valeyellow46, ]|               |
|     [marcmarquez93]|  marcmarquez93|
|[jessansan, 88jor...|      jessansan|
|[jessansan, 88jor...|  88jorgemartin|
|[jessansan, 88jor...|pitoslocosmclub|
|[jessansan, 88jor...|circuitodejerez|
|[jessansan, 88jor...|    luissalom39|
|[jessansan, 88jor...|      jessansan|
|[jessansan, 88jor...|  88jorgemartin|
|[jessansan, 88jor...|pitoslocosmclub|
+--------------------+---------------+
only showing top 20 rows



Si realizamos un count() sobre la columna m agrupando por valores distintos, tenemos resuelta la consulta.

In [26]:
metions.groupBy('m').agg(count('m').alias('y')).orderBy('y',ascending=False).limit(5).show()

+--------------+-----+
|             m|    y|
+--------------+-----+
|  valeyellow46|19599|
| marcmarquez93|16826|
|        motogp| 7770|
|     lorenzo99| 6969|
|26_danipedrosa| 3093|
+--------------+-----+



Estas son las 5 cuentas más mencionadas. Como era de esperar todas ellas se refieren a pilotos (la primera cuenta es la de Valentino Rossi) y una a la competición motogp. 

# CONSULTA 4

Calculamos primero los mensajes con más retweets y con más respuestas, más tarde en el segundo paso aplicaremos la ventana temporal indicada en el enunciado.

Calculamos los mensajes con más RT, para ello tenemos que filtrar por tipe_action = RT y agrupar por el Body del tweet.

In [27]:
Parte1_ret = events.filter(events.Tipe_action == 'RT').select('Body', 'Tipe_action')

In [28]:
Parte1_ret.show()

+--------------------+-----------+
|                Body|Tipe_action|
+--------------------+-----------+
| mbah goolge terc...|         RT|
|' rt @motogp: ano...|         RT|
|!!! rt @dhika46: ...|         RT|
|!!!! rt @birtymot...|         RT|
|#46 never die rt ...|         RT|
|#46 rt @motogp: c...|         RT|
|#93 rt @hndra_wir...|         RT|
|#99 #viscabarca :...|         RT|
|#akurapopo rt @lo...|         RT|
|#akurapopo rt @mo...|         RT|
|#animojorge rt @o...|         RT|
|#barcaday & #marq...|         RT|
|#bleedingnose rt ...|         RT|
|#btsport @btsport...|         RT|
|#cal #brad #smith...|         RT|
|#dp26 at qatar mo...|         RT|
|#elclasicomotogp ...|         RT|
|#emozioneda93 rt ...|         RT|
|#ferrari #f1 rt @...|         RT|
|#forzavale #goval...|         RT|
+--------------------+-----------+
only showing top 20 rows



Una vez tenemos filtrados los mensajes que nos interesan agrupamos y contamos por el campo Body.

In [29]:
part2_ret = (Parte1_ret.groupBy('Body')
                       .agg(count('Body').alias('numero'))
                       .orderBy('numero', ascending=False)
                        .limit(10))

In [30]:
part2_ret.show()

+--------------------+------+
|                Body|numero|
+--------------------+------+
|rt @marcmarquez93...|  5538|
|rt @valeyellow46:...|  4995|
|rt @marcmarquez93...|  3320|
|rt @marcmarquez93...|  3237|
|rt @motogp: #qata...|  2874|
|rt @valeyellow46:...|  2407|
|rt @motogp: anoth...|  2306|
|rt @marcmarquez93...|  1773|
|rt @lorenzo99: pe...|  1690|
|rt @motogp: for t...|  1640|
+--------------------+------+



part2_ret contiene el numero de veces que cada mensaje ha sido RT.

Continuamos nuestro estudio con los tweets con más respuestas. Nos encontramos en una situación similar a la anterior, debemos fija que tipe_action!= RT y que el campo Parent_sys_id sea != 'sin_padre' con esta serie de pasos nos aseguramos que estamos filtrando los tweets que deseamos 

In [31]:
Parte1_res = (events.select('Parent_sys_id' ,'Tipe_action')
                    .filter(events.Tipe_action != 'RT')
                     .filter(events.Parent_sys_id != 'sin padre')
                     .groupBy('Parent_sys_id')
                       .agg(count('Parent_sys_id').alias('numero'))
                       .orderBy('numero', ascending=False)
                        .limit(10))

In [32]:
Parte1_res.show()

+-------------+------+
|Parent_sys_id|numero|
+-------------+------+
|       681159|   554|
|       605727|   530|
|       603804|   519|
|       674825|   493|
|       460820|   235|
|       738975|   210|
|       758581|   154|
|       488086|   136|
|       461756|   126|
|       710916|   115|
+-------------+------+



Pasamos a la segunda parte del ejercicio, debemos repetir el tratamiento anterior pero fijando ahora una ventana temporal. Antes de comenzar revisamos que pinta tiene el campo Pub_date.

In [33]:
events.select('Pub_date').show()

+-------------------+
|           Pub_date|
+-------------------+
|2014-03-23 22:32:00|
|2014-03-23 22:48:00|
|2014-03-23 18:37:00|
|2014-03-24 01:54:00|
|2014-03-24 01:54:00|
|2014-03-24 01:54:00|
|2014-03-25 11:45:00|
|2014-03-24 13:29:00|
|2014-03-23 23:49:00|
|2014-03-23 21:54:00|
|2014-03-25 17:47:00|
|2014-03-23 21:13:00|
|2014-03-25 17:47:00|
|2014-03-23 19:11:00|
|2014-03-23 20:48:00|
|2014-03-23 19:24:00|
|2014-03-26 12:16:00|
|2014-03-24 13:03:00|
|2014-03-23 23:06:00|
|2014-03-23 21:06:00|
+-------------------+
only showing top 20 rows



Vamos a proceder al filtrado de la ventana temporal:

In [34]:
events4 = events.filter((events.Pub_date >= ("2014-03-24 05:00:00")) & (events.Pub_date <= ("2014-03-24 11:00:00")))

In [35]:
events4.select('Pub_date').orderBy('Pub_date', ascending=False).show()

+-------------------+
|           Pub_date|
+-------------------+
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
|2014-03-24 11:00:00|
+-------------------+
only showing top 20 rows



Ahora que ya tenemos fijada la ventana temporal y guardados los tweets que se realizaron dentro de ella, vamos a realizar sobre esta los calculos anteriores.

In [36]:
Parte2_res = (events4.select('Parent_sys_id' ,'Tipe_action')
                    .filter(events.Tipe_action != 'RT')
                     .filter(events.Parent_sys_id != 'sin padre')
                     .groupBy('Parent_sys_id')
                       .agg(count('Parent_sys_id').alias('numero'))
                       .orderBy('numero', ascending=False)
                        .limit(10))

In [37]:
Parte2_res.show()

+-------------+------+
|Parent_sys_id|numero|
+-------------+------+
|       603804|    65|
|       605727|    54|
|       666385|    30|
|       664891|    16|
|       645225|    14|
|       645626|    14|
|       645328|    13|
|       626650|     9|
|       622993|     8|
|       579001|     7|
+-------------+------+



In [38]:
Parte3_ret = events4.filter(events.Tipe_action == 'RT').select('Body', 'Tipe_action')

In [39]:
part4_ret = (Parte3_ret.groupBy('Body')
                       .agg(count('Body').alias('numero'))
                       .orderBy('numero', ascending=False)
                        .limit(10))

In [40]:
part4_ret.show()

+--------------------+------+
|                Body|numero|
+--------------------+------+
|rt @marcmarquez93...|   475|
|rt @jessansan: no...|   445|
|rt @jessansan: qu...|   390|
|rt @marcmarquez93...|   333|
|rt @jessansan: va...|   222|
|rt @lorenzo99: pe...|   167|
|rt @falcio78: @va...|   148|
|rt @albitebaldi: ...|   147|
|rt @motogp: #qata...|   101|
|rt @mflamigni: ch...|    97|
+--------------------+------+

